In [1]:
from pennylane import numpy as np
import pennylane as qml

In [2]:
dev = qml.device("default.qubit", wires=2, shots=1)

@qml.qnode(dev)
def guess_the_unitary(unitary):
    """Given a unitary that performs a Z or a -Z operation
    on a qubit, guess which one it is.
    
    Args: 
        U (array[complex]): A unitary matrix, guaranteed to be either Z or -Z.
    
    Returns:
        array [int]:  Probabilities on  on the first qubit
        using qml.probs()
    """
    ##################
    # YOUR CODE HERE #
    ##################  
    qml.Hadamard(wires=0)
    qml.ctrl(qml.QubitUnitary, control=0)(unitary, wires=1)
    qml.Hadamard(wires=0)
    
    return qml.probs(wires=0) #qml.expval(qml.PauliZ(0))

# Z gate 
U = qml.PauliZ.matrix 

# -Z gate
U = (-1)*qml.PauliZ.matrix

print(guess_the_unitary(U))

[0. 1.]


In [4]:
U = qml.T._matrix()
print(U)

[[1.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]


In [9]:
def U_power_2k(u,k):
    return np.linalg.matrix_power(u, 2**k)

In [6]:
U**2

array([[1.00000000e+00+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 2.22044605e-16+1.j]])

In [14]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def apply_controlled_powers_of_U(unitary):
    for k in estimation_wires:
        actual_k = len(estimation_wires) - k # because shit is in weird unclear reverse order
        print(f'control :{actual_k}')
        print(U_power_2k(unitary, actual_k))
        print(f'target :{target_wires}')
        qml.ctrl(qml.QubitUnitary, control=k)(U_power_2k(unitary, actual_k), wires=target_wires)


In [15]:
apply_controlled_powers_of_U(U)

control :3
[[1.+0.0000000e+00j 0.+0.0000000e+00j]
 [0.+0.0000000e+00j 1.-8.8817842e-16j]]
target :[3]
control :2
[[ 1.+0.0000000e+00j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j -1.+4.4408921e-16j]]
target :[3]
control :1
[[1.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 2.22044605e-16+1.j]]
target :[3]


In [21]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def estimate_phase(probs):
    """Estimate the value of a phase given measurement outcome probabilities
    of the QPE routine.
    
    Args: 
        probs (array[float]): Probabilities on the estimation wires.
    
    Returns:
        float: the estimated phase   
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    most_likely_value = np.argmax(probs) #or do we do a weighted average where the probs are the weights?
    print(most_likely_value)
    bits = [int(x) for x in np.binary_repr(most_likely_value, width=len(estimation_wires))]
    print(bits)
    bin_fraction = 0
    for k,b in enumerate(bits):
        bin_fraction += b / 2**(k+1)
        
    return bin_fraction

In [22]:
ps = [1.3479e-32, 1.0000e+00, 2.7843e-32, 2.4652e-32, 3.3868e-32, 5.6093e-32,
8.4698e-32, 2.6193e-31]

In [23]:
estimate_phase(ps)

1
[0, 0, 1]


0.125